# Fluid-solid reflection and transmission at a planar interface

In this notebook we consider a fluid over a solid half space. We adopt the same geometric setting as before. The family of waves we need to consider are the following:
    
**Incoming P wave**:

\begin{equation}
\mathbf{u}=\begin{pmatrix} \sin\psi_1 \\ 0 \\ -\cos\psi_1 \end{pmatrix} e^{i(\mathbf{k}\cdot\mathbf{x}-\omega t)}\,,\quad \mathbf{k}=\frac{\omega}{\alpha_1} \begin{pmatrix} \sin\psi_1 \\ 0 \\ -\cos\psi_1 \end{pmatrix}
\end{equation}

Note that we use $\phi$ for S wave angles and $\psi$ for P wave angles. In any case, the set of reflected and transmitted waves is always the same:

**Reflected P wave**:

\begin{equation}
\mathbf{u}=R_P\begin{pmatrix} \sin\psi_1 \\ 0 \\ \cos\psi_1 \end{pmatrix} e^{i(\mathbf{k}\cdot\mathbf{x}-\omega t)}\,,\quad \mathbf{k}=\frac{\omega}{\alpha_1} \begin{pmatrix} \sin\psi_1 \\ 0 \\ \cos\psi_1 \end{pmatrix}
\end{equation}

Note that the polarisation vector is defined such that the polarisation/amplitude of the incident and reflected SC waves are identical for normal incidence ($\phi_1=0$).

**Transmitted SV wave**:

\begin{equation}
\mathbf{u}=T_S \begin{pmatrix} \cos\phi_2 \\ 0 \\ \sin\phi_2 \end{pmatrix} e^{i(\mathbf{k}\cdot\mathbf{x}-\omega t)}\,,\quad \mathbf{k}=\frac{\omega}{\beta_2} \begin{pmatrix} \sin\phi_2 \\ 0 \\ -\cos\phi_2 \end{pmatrix}
\end{equation}

**Transmitted P wave**:

\begin{equation}
\mathbf{u}=T_P \begin{pmatrix} \sin\psi_2 \\ 0 \\ -\cos\psi_2 \end{pmatrix} e^{i(\mathbf{k}\cdot\mathbf{x}-\omega t)}\,,\quad \mathbf{k}=\frac{\omega}{\alpha_2} \begin{pmatrix} \sin\psi_2 \\ 0 \\ -\cos\psi_2 \end{pmatrix}
\end{equation}

The application of the kinematic and dynamic boundary conditions leads to a linear system of three equations, that involves the scattering matrix and the vector or reflection/transmission coefficients:

\begin{equation}
\begin{pmatrix}
\cos\psi_1 & \cos\psi_2 & - \sin\phi_2 \\\\
-\lambda_1\alpha_1^{-1} & (1+\cos^2\psi_2) \lambda_2\alpha_2^{-1} & - \mu_2\beta_2^{-1} \sin\psi_2\cos\phi_2 \\\\
0 & 2\alpha_2^{-1} \sin\psi_2\cos\psi_2 & \beta_2^{-1} (\cos^2\phi_2 - \sin^2\phi_2)
\end{pmatrix}
\begin{pmatrix} R_P \\ T_P \\ T_S \end{pmatrix} = \mathbf{h}
\end{equation}

The right-hand side is

\begin{equation}
\mathbf{h} = \begin{pmatrix} \cos\psi_1 \\ \lambda_1\alpha_1^{-1} \\ 0 \end{pmatrix}
\end{equation}


# 0. Python packages and input

## 0.1. Python packages and figure embellishment

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams["font.family"] = "Arial"
plt.rcParams.update({'font.size': 20})
plt.rcParams['xtick.major.pad']='12'
plt.rcParams['ytick.major.pad']='12'

## 0.2. Input parameters 

In [ ]:
# P velocity of upper acoustic layer [m/s].
alpha1=1500.0
# Density of upper acoustic layer [kg/m**3].
rho1=1000.0

# P velocity of elastic half space [m/s].
alpha2=1510.0
# S velocity of elastic half space [m/s].
beta2=50.0
# Density of elastic half space [kg/m**3].
rho2=1100.0

In [ ]:
# Compute elastic moduli.
lambda1=rho1 * alpha1**2 
mu2=rho2 * beta2**2
lambda2=rho2 * alpha2**2  - 2.0*mu2

In [ ]:
# Compute critical angle.
phi_cp=np.arcsin(alpha1/alpha2)*180.0/np.pi
print('P-wave critical angle: %f degree' % phi_cp)
phi_cs=np.arcsin(alpha1/beta2)*180.0/np.pi
print('S-wave critical angle: %f degree' % phi_cs)

# 1. Reflection and transmission coefficients

## 1.1. Compute and reflection and transmission coefficients

In [ ]:
def scattering_matrix(phi_pi,alpha1,rho1,alpha2,beta2,rho2):
    
    # Compute elastic moduli.
    lambda1=rho1 * alpha1**2 
    mu2=rho2 * beta2**2
    lambda2=rho2 * alpha2**2  - 2.0*mu2
    
    # Compute sines and cosines of all other angles.
    
    sin_phi_pt=(alpha2/alpha1)*np.sin(phi_pi)
    sin_phi_st=(beta2/alpha1)*np.sin(phi_pi)

    cos_phi_pt=np.sqrt(1.0-sin_phi_pt**2 + 0.0j)
    cos_phi_st=np.sqrt(1.0-sin_phi_st**2 + 0.0j)
    
    # Compute entries of the scattering matrix.
    A11=np.cos(phi_pi)
    A12=cos_phi_pt
    A13=-sin_phi_st

    A21=-1.0
    A22=((alpha1*lambda2)/(alpha2*lambda1))*(1.0+cos_phi_pt**2)
    A23=-((mu2*alpha1)/(lambda1*beta2))*sin_phi_st*cos_phi_st

    A31=0.0
    A32=2.0*sin_phi_pt*cos_phi_pt
    A33=(alpha2/beta2)*(cos_phi_st**2-sin_phi_st**2)

    A=np.array([[A11, A12, A13],[A21, A22, A23],[A31, A32, A33]])
    
    # Return.
    return A

In [ ]:
phi_pi=np.linspace(0.0,np.pi/2.0-0.01,10000)
Rp_array=np.zeros(len(phi_pi),dtype='complex64')
Tp_array=np.zeros(len(phi_pi),dtype='complex64')
Ts_array=np.zeros(len(phi_pi),dtype='complex64')

for i in range(len(phi_pi)):
    
    # Compute scattering matrix.
    A=scattering_matrix(phi_pi[i],alpha1,rho1,alpha2,beta2,rho2)
    # Compute right-hand side of scattering equation.
    b=np.array([[np.cos(phi_pi[i])],[1.0],[0.0]])
    # Solve equation.
    Rp,Tp,Ts=np.matmul(np.linalg.inv(A),b)
    Rp_array[i]=Rp
    Tp_array[i]=Tp
    Ts_array[i]=Ts

## 1.2. Plot as a function of incidence angle

In [ ]:
fig=plt.figure(figsize=(10,10))
plt.plot(phi_pi*180.0/np.pi,np.real(Rp_array),'b',linewidth=2)
plt.plot(phi_pi*180.0/np.pi,np.imag(Rp_array),'r',linewidth=2)
plt.plot(phi_pi*180.0/np.pi,np.abs(Rp_array),'k',linewidth=3)
plt.minorticks_on()
plt.grid(which='major',color='k',linewidth=1.0)
plt.grid(which='minor',color='k',linewidth=0.5)
plt.xlabel('incidence angle [deg]')
plt.ylabel('Rp')
plt.xlim([0.0,90.0])
#plt.ylim([0.8,1.05])
plt.show()

fig=plt.figure(figsize=(10,10))
plt.plot(phi_pi*180.0/np.pi,np.real(Tp_array),'b',linewidth=2)
plt.plot(phi_pi*180.0/np.pi,np.imag(Tp_array),'r',linewidth=2)
plt.plot(phi_pi*180.0/np.pi,np.abs(Tp_array),'k',linewidth=3)
plt.minorticks_on()
plt.grid(which='major',color='k',linewidth=1.0)
plt.grid(which='minor',color='k',linewidth=0.5)
plt.xlabel('incidence angle [deg]')
plt.ylabel('Tp')
plt.xlim([0.0,90.0])
plt.show()

fig=plt.figure(figsize=(10,10))
plt.plot(phi_pi*180.0/np.pi,np.real(Ts_array),'b',linewidth=2)
plt.plot(phi_pi*180.0/np.pi,np.imag(Ts_array),'r',linewidth=2)
plt.plot(phi_pi*180.0/np.pi,np.abs(Ts_array),'k',linewidth=3)
plt.minorticks_on()
plt.grid(which='major',color='k',linewidth=1.0)
plt.grid(which='minor',color='k',linewidth=0.5)
plt.xlabel('incidence angle [deg]')
plt.ylabel('Ts')
plt.xlim([0.0,90.0])
plt.show()